
# 영화추천 시스템

**1. 간단한 추천 엔진:**

> TMDB 투표 수와 투표 평균을 활용하여 전체 및 특정 장르의 Top Movies Charts 생성

> IMDB 가중 등급 시스템을 사용하여 영화의 등급을 계산하며, 최종적으로 정렬 및 추천에 사용

> 인기 있는 영화를 추천하는 데 좋은 시작점이며 대중적임

**2. 콘텐츠 기반 추천 엔진:**

> 2-1. 영화 개요 및 태그라인 활용하여, 예측영화 내용을 이해하기 위해 자연어 처리 또는 텍스트 분석을 사용

> 2-2. 출연자, 스태프, 장르 및 키워드와 같은 메타데이터를 고려하여 예측하고,  투표 수와 높은 평점과 같은 요소를 고려

**3. 협업 필터링**


> Surprise 라이브러리를 사용하여 특이 값 분해를 기반으로 하는 협업 필터링 접근 방식을 구현

> 협업 필터링은 사용자가 지난 번에 비슷한 항목을 좋아했을 경우 미래에도 비슷한 항목을 좋아할 것으로 기대하는 아이디어에 착안

> 낮은 평균 제곱근 오차 (RMSE)를 달성했으며 추천 엔진의 성능을 나타내는 좋은 지표

**4. 하이브리드 엔진**

> 콘텐츠 기반 및 협업 필터링에서 각각의 요소를 결합하여 영화 추천을 제공

> 특정 사용자를 위해 내부적으로 계산한 예상 등급을 기반으로 사용자에게 영화 제안을 제공

# ■ 요약

- 간단한 단순 추천 시스템, 개인화 추천이 가능한 콘텐츠 기반 시스템,
  사용자와 사용자 기록/아이템과의 상호적인 작용에 따른 협업 필터링 시스템 3가지를 살펴본다.

- 먼저, 간단한 단순 추천 시스템을 설명하고, 개인화하여 추천하지 못하는
  단순 추천 시스템의 한계점을 살펴 본다.

- 이 다음, 콘텐츠 기반 추천 시스템을 살펴 본다.
  - 라이브러리 및 알고리즘: Surprise 라이브러리와 SVD 알고리즘을 사용.
  - 적용 방법: 사용자-아이템 평점 데이터에 SVD를 적용하여 모델 학습.
  - 결과: 특정 사용자에 대한 아이템의 예상 평점을 예측.

- 세번째로 협업 필터링 추천 시스템을 살펴 본다.
  - 라이브러리 및 알고리즘: Surprise 라이브러리와 SVD 알고리즘을 사용.
  - 적용 방법: 사용자-아이템 평점 데이터에 SVD를 적용하여 모델 학습.
  - 결과: 특정 사용자에 대한 아이템의 예상 평점을 예측.

- 마지막으로, 하이브리드 추천을 살펴 본다.

  - 협업 필터링으로 특정 사용자에게 높은 예상 평점을 받을 것으로       
예측되는 영화 목록 추출.
  -  해당 영화 목록을 콘텐츠 기반 필터링의 결과로 정렬. 특히, LDA로    
추출한 토픽 정보를 사용하여 콘텐츠의 유사도를 판단.
  - 최종적으로 사용자에게 특정 영화와 콘텐츠가 유사하며 동시에 사용자의 선호도가 높은 영화를 추천.


In [ ]:
#데이터 불러오기
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#서프라이즈

!pip install scikit-surprise
!pip install --upgrade scikit-surprise
!pip install pandas

In [ ]:

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
import warnings
warnings.simplefilter('ignore')

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
# evaluate 실행 안되서 추가

import warnings; warnings.simplefilter('ignore')

#Movies Recommender System ; Movie Data Analysis and Recommendation Systems

Movies Recommender System ; Movie Data Analysis and Recommendation Systems

■ 목표 :

 몇 가지 추천 알고리즘(콘텐츠 기반, 인기 기반 및 협업 필터링)을 구현하고 이러한 모델의 앙상블을 구축하여 최종 추천 시스템을 마련


■ 데이터 세트의 전체 내용

 1) 전체 데이터 세트: 270,000명의 사용자가 45,000편의 영화에 적용한 26,000,000개의 평가와 750,000개의 태그 애플리케이션으로 구성.

  1,100개의 태그에 걸쳐 1,200만 개의 관련성 점수가 포함된 태그 데이터가 포함

 2) 소규모 데이터 세트: 700명의 사용자가 9,000편의 영화에 적용한 100,000개의 평가와 1,300개의 태그 애플리케이션으로 구성

# 1. Simple Recommender

cf. simple recommender 설명

영화의 인기도와 장르를 바탕으로 추천시스템을 제공

기본 아이디어  더 인기가 높고 비평가가 호평하는 영화 --> 일반 관람객도 좋아함

이는 사용자 기반으로 데이터를 활용한 추천 시스템은 아님

1 평점 및 인기 -> 영화 정렬 -> 상위 영화를 표시

2 추가로 장르별 인기 영화를 표시

### cf. 컬럼 명
/성인 영화 여부
/컬렉션 소속
/예산
/장르
/홈페이지
/ID
/IMDB ID
/원래 언어
/원제목
/개요
/출시 날짜
/수익
/상영 시간
/대화하는 언어
/상태
/태그라인
/제목
/비디오
/평균 투표 점수
/투표 수

In [ ]:
md = pd. read_csv('/content/drive/My Drive/Colab Notebooks/movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
# 장르를 쉽게 검색하기 위해, 문자열을 리스트로 변경
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

cf. 영화 순위 구하기 및 장르 차트 만들기

■ Top Movies Chart
: 이 공식은 단순히 영화의 평균 평점만을 기준으로 순위를 정하는 것이 아니라,

투표 수와 평균 평점을 함께 고려하여 영화를 평가.

이렇게 하는 이유는 단순히 점수로 평가를 하는 것이 아닌,

적은 투표수보다 많은 투표 수를 받은 영화에 가중치를 두어 계산


■ Weighted Rating (WR) = (vv+m.R)+(mv+m.C)

v : 투표수

m : 차트에 리스트업 가능한 최소 투표수

R : 평균 등급

C : 전체 평균 투표수

m 값 : 95 백분위수를 활용할 예정임

즉, 차트에 리스트업 되려면 투표수가 전체 상위 5% 안에는 들어야함

■ Top 250 Chart 차트를 만들기

■ 특정 장르의 차트 만들기 기능을 정의

In [ ]:
#### md[md['vote_count'].notnull()]: md라는 데이터프레임에서 vote_count 컬럼의 값이 null(빈 값)이 아닌 행만을 선택
#### ['vote_count']: 위에서 선택된 행들 중에서 vote_count 컬럼만을 선택
#### .astype('int'): 선택된 vote_count 컬럼의 값들을 정수(int) 형태로 변환
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')

#### md[md['vote_average'].notnull()]: md라는 데이터프레임에서 vote_count 컬럼의 값이 null(빈 값)이 아닌 행만을 선택
#### ['vote_average']: 위에서 선택된 행들 중에서 vote_count 컬럼만을 선택
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [ ]:
m = vote_counts.quantile(0.95)
m

434.0

cf. vote_counts 데이터에서 상위 5%에 해당하는 투표수의 기준 값을 계산

.quantile(0.95)는 해당 데이터에서 95번째 백분위수를 구하는 메서드

vote_counts 데이터에서 상위 5%(100% - 95% = 5%)의 값들 중 가장 낮은 값을 반환

결과적으로, m의 값은 434이며,

이는 vote_counts 데이터에서 상위 5%에 속하는 영화들의 투표수가 최소 434번 이상

In [ ]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [ ]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

최소 득표수 434표

평균 등급은 5등급

10점 만점에 5.244점

2274개의 영화가 차트에 들어옴


In [ ]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

##.apply(): Pandas의 DataFrame에 함수를 적용하는 메서드.
## weighted_rating: 앞서 정의한 가중 평점을 계산하는 함수
## axis=1: 함수를 각 행(row)에 적용하라는 의미입니다. axis=0일 경우 각 열(column)에 적용
## 결과적으로, qualified 데이터프레임에는 각 영화의 가중 평점을 나타내는 새로운 컬럼 'wr'이 추가

In [ ]:
qualified = qualified.sort_values('wr', ascending=False).head(250)
##cf. 각 영화의 가중 평점을 나타내는 wr 행을 오름차순으로 250개 보기


Top Movies

In [ ]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


### cf. 중간 결과 및 분석 방법 변경

크리스토퍼 놀란의 세 편의 영화인 인셉션(Inception), 다크 나이트(The Dark Knight), 인터스텔라(Interstellar)가 차트 상단에 존재

### 95백분위수 - > 85 백분위수로 변경

특정장르에 대한 차트를 위해서 95백분위수가 아닌 85백분위수로 변경하여 여러 장르를 살펴봄.

인기 없는 장르는 투표 수가 적을 수도 있고, 데이터의 시각화를 통한 다양성을 확인 가능

In [ ]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

cf.장르를 분리하고 병합

장르를 분리한 후 병합하는 과정은 특정 장르에 대한

차트를 더 정확하고 다양하게 만들기 위한 목적

In [ ]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)

    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')

    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)

    return qualified

cf. 중간 결과 및 해석

상위 15개 로맨스 영화
(전체 기준으로 로맨스가 없음)

□함수의 내부 동작 해석

- 장르에 해당하는 영화 선택:

입력받은 장르(genre)에 해당하는 영화만 gen_md 데이터프레임에서 선택하여 df에 저장

- 투표수 및 평균 투표 점수 계산: 장르에 해당하는 영화들 중 투표수(vote_count)와 평균 투표 점수(vote_average)가 null이 아닌 영화들만 선택하여 각각의 평균값을 계산

- 백분위수 기준 투표수 계산: 해당 장르의 영화들의 투표수 중에서 입력받은 백분위수(percentile)에 해당하는 투표수를 m에 저장

- 차트에 들어갈 영화 선정: m보다 많은 투표를 받은 영화들 중에서 투표수와 평균 투표 점수가 null이 아닌 영화들만 선택하여 qualified에 저장

- 가중 평균 점수 계산: 선택된 영화들(qualified)에 대해 가중 평균 점수를 계산하여 wr 컬럼에 저장

- 가중 평균 점수 순으로 정렬: 계산된 가중 평균 점수(wr)를 기준으로 qualified를 내림차순으로 정렬

- 상위 250개 영화 선택: 가중 평균 점수가 높은 순서대로 상위 250개의 영화만 선택하여 반환

In [ ]:
##상위 15개 로맨스 영화
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


In [ ]:
##상위 15개 애니메이션 영화
build_chart('Animation').head(15)

,title,year,vote_count,vote_average,popularity,wr
359,The Lion King,1994,5520,8,21.605761,7.909339
5481,Spirited Away,2001,3968,8,41.048867,7.875933
9698,Howl's Moving Castle,2004,2049,8,16.136048,7.772103
2884,Princess Mononoke,1997,2041,8,17.166725,7.771305
5833,My Neighbor Totoro,1988,1730,8,13.507299,7.735274
40251,Your Name.,2016,1030,8,34.461252,7.589820
5553,Grave of the Fireflies,1988,974,8,0.010902,7.570962
19901,Paperman,2012,734,8,7.198633,7.465676
39386,Piper,2016,487,8,11.243161,7.285132
20779,Wolf Children,2012,483,8,10.249498,7.281198


# 2. Content Based Recommender

Simple Recommender는 개인 선호도와 관계없이 동일한 추천을 제공한다는 한계점이 있음

simple recommender의 한계 및 추천 알고리즘 다른 방안 제시


■목표 : 영화간의 유사성을 계산하고, 개인이 좋아하는 특정 영화와 가장 유사한 영화를 제안하는 시스템

<콘텐츠 기반 필터링>

영화 개요 및 태그라인, 영화 출연진, 제작진, 키워드 및 장르를 기반으로 두 개의 콘텐츠 기반 추천시스템을 구축

In [ ]:
links_small = pd.read_csv('/content/drive/My Drive/Colab Notebooks/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
# tmdbld만 불러오기

cf. 컬럼 구성

movieId: 특정 데이터베이스나 응용 프로그램에서 해당 영화를 참조하기 위해 사용

imdbId: IMDb는 영화, 텔레비전 쇼, 연예인, 크루 멤버 등에 대한 정보를 제공하는 온라인 데이터베이스

tmdbId: TMDb는 IMDb와 유사한 온라인 영화 및 TV 쇼 데이터베이스

tmdbId는 TMDb 웹사이트에서 해당 영화의 페이지에 직접 연결 가능

In [ ]:
md = md.drop([19730, 29503, 35587])

In [ ]:
##id를 정수 타입으로 변환

md['id'] = md['id'].astype('int')

In [ ]:
#md 데이터프레임에서 links_small에 나열된 tmdbId에 해당하는 영화들만을 추출하여 smd 데이터프레임에 저장하는 작업

smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

 9099개의 영화를 사용함

 기존 데이터 4.5만개보다는 훨씬 작은 수치

 links_small을 사용하는 주요 이유는 전체 데이터셋에서 작은 서브셋(subset)을 활용하여 모델링이나 다른 작업을 진행하기 위함 특히 영화 추천 시스템의 경우, 전체 영화 대신 주요하거나 관련된 일부 영화를 대상으로 한 추천을 진행



# 1. 영화 설명 및 태그라인 활용


 Movie Description Based Recommender

In [ ]:
credits = pd.read_csv('/content/drive/My Drive/Colab Notebooks/credits.csv')
keywords = pd.read_csv('/content/drive/My Drive/Colab Notebooks/keywords.csv')

In [ ]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [ ]:
#용어빈도수 확인

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

cf. TF-IDF란 무엇인가?

TF-IDF는 특정 문서에 있는 단어의 중요도를 나타내주는 기법

예를 들어, "영화"라는 단어가 많은 영화 설명에 고르게 나타나면 그 단어는 특별하지 않음.

하지만 "토이스토리"라는 단어는 "토이스토리"라는 영화 설명에서만 자주 나타날 것이다.

TF-IDF는 이런 차이를 계산

cf. tfidf_matrix의 의미는 무엇인가?

smd['description']는 영화 설명

tfidf_matrix는 이 영화 설명들을 숫자로 바꿔놓은 표 같은 것. 이 표를 사용하면 영화 간의 유사도를 쉽게 알 수 있음

In [ ]:
tfidf_matrix.shape

(9099, 268124)

cf. tfidf_matrix.shape가 (9099, 268124)의 의미는?


총 9099개의 영화 설명이 있음

268124은 영화 설명에서 발견된 모든 단어와 단어의 조합

(2개의 단어가 연속으로 나타나는 경우)의 총 개수이다.

결론적으로, 우리는 TF-IDF를 사용해서 영화 설명들을 숫자로 바꾼 표를 만들었고, 이 표를 사용해서 어떤 영화들이 서로 유사한지 확인가능

코사인 유사성을 활용하여 두 영화간 유사성을 계산



In [ ]:
##cosine_sim은 각 영화와 다른 영화들 간의 유사도
## linear_kernel은 코사인 유사도를 계산할 때 사용하는 함수로
## cosine_sim에는 각 영화와 다른 영화들 간의 유사도가 저장

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
### 첫번째 영화와 모든 영화들의 유사도 점수를 보기

cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

영화간 유사성 행렬
가장 유사 영화 30편 반환 행렬 만들기


In [ ]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

'The Godfather' 해보기

In [ ]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

다크나이트 해보기

In [ ]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

In [ ]:
get_recommendations('My Neighbor Totoro').head(10)

3504    Final Fantasy: The Spirits Within
7744                      Red Riding Hood
5749                         The Landlord
8821                          Poltergeist
7755                            Insidious
4951                 What's New Pussycat?
8199                      Our Little Girl
3787                       Female Trouble
1596             The Cat from Outer Space
1219                         Fathers' Day
Name: title, dtype: object

영화의 등급과 인기를 결정하는 출연진, 제작진, 감독 및 장르와 같은 매우 중요한 기능을 고려하지 않다는 한계점 존재


# 2.장르, 키워드, 출연진 및 제작진 활용

Metadata Based Recommender

In [ ]:
#데이터 변환하기

keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [ ]:
#행렬수 확인하기

md.shape

(45463, 25)

In [ ]:
#데이터 병합

md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [ ]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

제작진: 감독만 선택

등장인: 크레딧 목록에 나오는 3인

In [ ]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
smd['director'] = smd['crew'].apply(get_director)

In [ ]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [ ]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


My approach to building the recommender is going to be extremely hacky. What I plan on doing is creating a metadata dump for every movie which consists of genres, director, main actors and keywords.
장르, 감독, 주인공, 키워드를 포함한 메타데이터 만들고

Count Vectorizer를 사용하여 앞에서 한것 처럼
매트릭스를 구성함

코사인유사성을 활용하여 계산



Strip Spaces and Convert to Lowercase from all our features. This way, our engine will not confuse between Johnny Depp and Johnny Galecki.
Mention Director 3 times to give it more weight relative to the entire cast.

장르, 감독, 주인공, 키워드를 활용해서

In [ ]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

Keywords

전처리 : 키워드 빈도수

In [ ]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [ ]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

1회 이상 제거

어간 분석(Dogs and Dog)

In [ ]:
s = s[s > 1]

In [ ]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [ ]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [ ]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

# 실행안됨, 아래 코드로 대체

In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

다크나이트

In [ ]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

In [ ]:
get_recommendations('Mean Girls').head(10)

3319               Head Over Heels
4763                 Freaky Friday
1329              The House of Yes
6277              Just Like Heaven
7905         Mr. Popper's Penguins
7332    Ghosts of Girlfriends Past
6959     The Spiderwick Chronicles
8883                      The DUFF
6698         It's a Boy Girl Thing
7377       I Love You, Beth Cooper
Name: title, dtype: object

In [ ]:
get_recommendations('Avatar').head(10)

974                             Aliens
522         Terminator 2: Judgment Day
1011                    The Terminator
922                          The Abyss
4347    Piranha Part Two: The Spawning
344                          True Lies
1376                           Titanic
8401           Star Trek Into Darkness
3216                Dungeons & Dragons
8724                 Jupiter Ascending
Name: title, dtype: object

Popularity and Ratings

앞에 적용한 추천시스템은 영화의 평점이나 인기와는 무관함

예시) '배트맨 앤 로빈', '다크 나이트'를 비교

인기있고 좋은 평가를 받은 영화만 추천하는 시스템으로 구상

유사성 점수에 기반하여 상위 25개의 영화를 선택하고, 60번째 백분위 평점을 계산

그 후, 이 값을 'm'의 값으로 사용하여 IMDB의 공식을 이용하여 각 영화의 가중 평점을 계산(Simple Recommender 처럼)

In [ ]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [ ]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
1134,Batman Returns,1706,6,1992,5.846862
132,Batman Forever,1529,5,1995,5.054144
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943
1260,Batman & Robin,1447,4,1997,4.287233


In [ ]:
improved_recommendations('Mean Girls')

,title,vote_count,vote_average,year,wr
1547,The Breakfast Club,2189,7,1985,6.709602
390,Dazed and Confused,588,7,1993,6.254682
8883,The DUFF,1372,6,2015,5.818541
3712,The Princess Diaries,1063,6,2001,5.781086
4763,Freaky Friday,919,6,2003,5.757786
6277,Just Like Heaven,595,6,2005,5.681521
6959,The Spiderwick Chronicles,593,6,2008,5.680901
7494,American Pie Presents: The Book of Love,454,5,2009,5.119690
7332,Ghosts of Girlfriends Past,716,5,2009,5.092422
7905,Mr. Popper's Penguins,775,5,2011,5.087912


'배트맨 앤 로빈', '다크 나이트' 둘다나옴

이건 TMDB에서 4점을 받았기 때문

하이브리드 엔진시에 재 논의

# 3. Collaborative Filtering

메타데이터 수집: 각 영화에 대한 메타데이터 덤프를 만들고 장르, 감독, 주요 배우, 키워드와 같은 정보를 포함

텍스트 전처리: 공백을 제거하고 텍스트를 소문자로 변환하는 것은 표준 텍스트 전처리 단계

텍스트 데이터가 일관성, 대소문자 또는 공백 차이로 인한 유사한 용어 혼동을 방지

특성 가중치 부여: 감독을 여러 번 언급하여 전체 배우 대비 더 높은 가중치를 부여
이로써 감독의 역할을 추천에서 중요하게 고려하며,
영화의 스타일과 주제를 결정하는 데 중요

카운트 벡터화: 텍스트 데이터(장르, 배우, 키워드)를 숫자 형식으로 변환하기 위해 카운트 벡터화를 사용

텍스트 데이터를 유사성 계산에 사용할 수 있는 형식으로 변환

코사인 유사성: 카운트 행렬을 가지고 영화 간의 코사인 유사성을 계산

코사인 유사성은 두 영화 사이의 유사성을 측정하는 일반적인 방법

이 경우, 메타데이터에 대한 유사성을 찾아 비슷한 영화를 찾는것이 가능

코사인 유사성을 기반으로하며, 주어진 영화와 가장 유사한 영화를 추천

사인 유사성이 높을수록 메타데이터 측면에서 영화는 더 유사함

In [ ]:
from surprise import Reader
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.dataset import DatasetAutoFolds

reader = Reader()

In [ ]:
ratings = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

# 실행안됌, 아래 코드로 대체

AttributeError: ignored

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

# Reader 정의
reader = Reader(rating_scale=(1, 5))

평점 척도의 범위를 1에서 5로 설정

In [ ]:
# Load your data from a pandas DataFrame
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [ ]:
# SVD 알고리즘 초기화
svd = SVD()

# 교차검증으로 알고리즘 평가
results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# 결과값 확인하기
for measure in ['test_rmse', 'test_mae']:
    print(f'{measure}: {results[measure].mean()}')

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9002  0.8986  0.8965  0.8952  0.8929  0.8967  0.0025  
MAE (testset)     0.6942  0.6934  0.6879  0.6900  0.6876  0.6906  0.0027  
Fit time          1.96    1.57    1.55    1.54    1.57    1.64    0.16    
Test time         0.12    0.35    0.15    0.12    0.31    0.21    0.10    
test_rmse: 0.8966892452913381
test_mae: 0.6906147985554354


SVD는 협업 필터링 알고리즘 중 하나로, 행렬 분해를 기반으로 사용자와 아이템 간의 상호 작용을 분석하여 추천을 수행

In [ ]:
trainset = data.build_full_trainset()

# Fit (train) the SVD model on the training set
svd.fit(trainset)

In [ ]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_0, *['movieId'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_1, *['rating'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_2, *['timestamp'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plot(df, x_colname, y_colname, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(6 * figscale, 6 * figscale))
  df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha)
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plot(_df_3, *['movieId', 'rating'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_7, *['movieId', 'rating', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_8, *['movieId', 'count()', None], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_9, *['movieId'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_10, *['rating'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_11, *['timestamp'], **{})
chart

In [ ]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.791467363364246, details={'was_impossible': False})

영화 ID 302에 대한 추정 평점은 2.686

추천시스템이 영화 자체나 영화 내용과는 무관함

영화 ID를 기반으로 작동하며,

다른 사용자가 해당 영화를 어떻게 예측했는지를 기반으로 평점을 예측함

기존의 데이터 분석과 다른점?

기존 데이터 분석은 감독자, 추천수로 추천되어서, 비슷한 영화만 계속 추천이 가능하였으나

장르, 설명, 태그라인을 기준으로 조금 더 다양한 메타데이터 기반으로 구성함으로써

구성된 내용을 통해 좀 더 개인화되고 추상적인 영화를 다양한 형태로 추천 가능

# 4. Hybrid Engine

Input: User ID and the Title of a Movie

Output: Similar movies sorted on the basis of expected ratings by that particular user.

In [ ]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [ ]:
id_map = pd.read_csv('/content/drive/My Drive/Colab Notebooks/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

# 캐그

In [ ]:
indices_map = id_map.set_index('id')

In [ ]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [ ]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.096084
2014,Fantastic Planet,140.0,7.6,1973,16306,2.992113
974,Aliens,3282.0,7.7,1986,679,2.972954
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,2.937012
1011,The Terminator,4208.0,7.4,1984,218,2.907555
1376,Titanic,7770.0,7.5,1997,597,2.857891
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,2.844444
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.827494
922,The Abyss,822.0,7.1,1989,2756,2.804271
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.800397


In [ ]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,year,id,est
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.199805
1376,Titanic,7770.0,7.5,1997,597,3.171498
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.143500
2014,Fantastic Planet,140.0,7.6,1973,16306,3.106292
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.090976
4347,Piranha Part Two: The Spawning,41.0,3.9,1981,31646,3.055823
6084,Beastmaster 2: Through the Portal of Time,17.0,4.6,1991,27549,3.045539
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,3.021343
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.998277
4017,Hawk the Slayer,13.0,4.5,1980,25628,2.989465


하이브리드 추천 시스템에서는 동일한 영화이지만
사용자마다 다른 추천을 받음

그래서 더 개인화되고 특정 사용자에 맞게 조정